# Comparing various outer material configurations for ARF

We look at how including polymer layers of various types influences loss in ARF.  Note that this is mostly for finding location of modes and to give a general idea of changes, not for final determination of loss.  The latter requires convergence studies for each configuration.

In [1]:
from fiberamp.fiber.microstruct.pbg import ARF2

from ngsolve.webgui import Draw
import ngsolve as ng

importing NGSolve-6.2.2203-56-g47f483033


# Default case: Air in outer region

In [52]:
A = ARF2(name='fine_cladding', poly_core=True)

ModeSolver: Checking if mesh has required regions
Mesh has  21941  elements,  10989  points,  and  32929  edges.


In [53]:
Draw(A.index, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [54]:
center = 5.066
radius = .1        
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=1, niterations=10,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


Set NGSolve automatic PML with p= 1  alpha= 5 and thickness=2.000
Using FEAST to search for vector leaky modes in
circle of radius 0.1 centered at  5.066
assuming not more than 4 modes in this interval
System size: 109740  x  109740   Inverse type: umfpack

SpectralProj: Setting shifted trapezoidal rule on elliptical contour
SpectralProjNGR:   Making resolvent at z =  +5.137 +0.004j


KeyboardInterrupt: 

In [5]:
betas

array([3488401.01282438+0.00010111j, 3488401.28035741+0.00134324j])

In [6]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

# Only glass everywhere outside core:

Now that our outer refractive index is higher than the core index, the locations of the Z values switches: the beta values are always approximately equal to k_low = k0 * n_low,  but the Z^2 values equal beta^2 - n0^2, and now n0 is drastically different, so we need to make up for this. 

The formula is at the bottom of the next cell

In [62]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 30 / scaling
T_hard_polymer = 30 / scaling

T_outer = 15 / scaling
T_buffer = 10 / scaling

n0 = n_glass

outer_materials = [

#     {'material': 'soft_polymer',
#      'n': n_soft_polymer,
#      'T': T_soft_polymer,
#      'maxh': 2},

#     {'material': 'hard_polymer',
#      'n': n_hard_polymer,
#      'T': T_hard_polymer,
#      'maxh': 2},

#     {'material': 'buffer',
#      'n': n0,
#      'T': T_buffer,
#      'maxh': 2},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': .2}
]

center = A.L**2 * A.k**2 * (n0**2 - n_air**2) + 5.066

In [63]:
center

2937.54050746878

In [64]:
A = ARF2(name='fine_cladding', poly_core=True, outer_materials=outer_materials)


ModeSolver: Checking if mesh has required regions
Mesh has  21701  elements,  10918  points,  and  32618  edges.


In [65]:
Draw(A.V, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [21]:
center = center
radius = .1       
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=6,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


Set NGSolve automatic PML with p= 2  alpha= 5 and thickness=1.000
Using FEAST to search for vector leaky modes in
circle of radius 0.1 centered at  2937.54050746878
assuming not more than 4 modes in this interval
System size: 37275  x  37275   Inverse type: umfpack

SpectralProj: Setting shifted trapezoidal rule on elliptical contour
SpectralProjNGR:   Making resolvent at z = +2937.611 +0.004j
SpectralProjNGR:   Making resolvent at z = +2937.470 +0.004j
SpectralProjNGR:   Making resolvent at z = +2937.470 -0.004j
SpectralProjNGR:   Making resolvent at z = +2937.611 -0.004j

=========== Starting FEAST iterations ===========
Trying with 4 vectors:

 ITERATION 1 with 4 vectors and 4 left vectors
   During kernel cleaning abs(g):
   [9.7e-03 1.9e-03 2.2e-17 9.8e-16]
   Removing ew#[2]
   Real part of computed eigenvalues:
   [2937.54045842 2937.54058825 2941.18257693]
   Imag part of computed eigenvalues:
   [-1.05053426e-04 -9.28821391e-05  3.53940998e-01]
   Relative Hausdorff distance f

In [22]:
betas

array([3488400.32891887+0.06692238j, 3488400.24620905+0.05916888j])

In [23]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

# Only soft polymer, extending everywhere outside core and cladding.

In [81]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 10 / scaling
T_hard_polymer = 30 / scaling

T_outer = 15 / scaling
T_buffer = 30 / scaling

n0 = n_soft_polymer

outer_materials = [

    {'material': 'soft_polymer',
     'n': n_soft_polymer,
     'T': T_soft_polymer,
     'maxh': .04},

#     {'material': 'hard_polymer',
#      'n': n_hard_polymer,
#      'T': T_hard_polymer,
#      'maxh': 2},

#     {'material': 'buffer',
#      'n': n0,
#      'T': T_buffer,
#      'maxh': 2},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': .2}
]
center = A.L**2 * A.k**2 * (n0**2 - n_air**2) + 5.066

In [82]:
center

2946.8813917475427

In [83]:
A = ARF2(name='fine_cladding', poly_core=True, refine=0, outer_materials=outer_materials)


ModeSolver: Checking if mesh has required regions
Mesh has  46338  elements,  23250  points,  and  69587  edges.


In [84]:
Draw(A.V, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [31]:
 A.mesh.GetMaterials()

('core', 'fill_air', 'inner_air', 'glass', 'soft_polymer', 'Outer')

In [32]:
center = center
radius = .1      
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=1, niterations=6,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


Set NGSolve automatic PML with p= 2  alpha= 5 and thickness=1.000
Using FEAST to search for vector leaky modes in
circle of radius 0.1 centered at  2946.8813917475427
assuming not more than 4 modes in this interval
System size: 39063  x  39063   Inverse type: umfpack

SpectralProj: Setting shifted trapezoidal rule on elliptical contour
SpectralProjNGR:   Making resolvent at z = +2946.952 +0.004j
SpectralProjNGR:   Making resolvent at z = +2946.811 +0.004j
SpectralProjNGR:   Making resolvent at z = +2946.811 -0.004j
SpectralProjNGR:   Making resolvent at z = +2946.952 -0.004j

=========== Starting FEAST iterations ===========
Trying with 4 vectors:

 ITERATION 1 with 4 vectors and 4 left vectors
   During kernel cleaning abs(g):
   [6.8e-03 4.1e-03 2.4e-09 2.8e-14]
   Real part of computed eigenvalues:
   [2946.88134536 2946.88140551 2947.27399788 2949.56803783]
   Imag part of computed eigenvalues:
   [-1.32278087e-07 -9.57083408e-06 -8.01197235e-02  1.79686334e+00]
   Relative Hausdor

In [33]:
betas

array([3488400.23758381+0.00239059j, 3488400.30772047+0.00745624j])

In [34]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

# Soft and Hard Polymer, Hard polymer extending to infinity

In [107]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 30 / scaling
T_hard_polymer = 15 / scaling

T_outer = 15 / scaling
T_buffer = 30 / scaling

n0 = n_hard_polymer

outer_materials = [

    {'material': 'soft_polymer',
     'n': n_soft_polymer,
     'T': T_soft_polymer,
     'maxh': .06},

    {'material': 'hard_polymer',
     'n': n_hard_polymer,
     'T': T_hard_polymer,
     'maxh': .07},

#     {'material': 'buffer',
#      'n': n0,
#      'T': T_buffer,
#      'maxh': 2},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': .2}
]
center = A.L**2 * A.k**2 * (n0**2 - n_air**2) + 5.066

In [108]:
A = ARF2(name='fine_cladding', poly_core=True, refine=0, outer_materials=outer_materials)


ModeSolver: Checking if mesh has required regions
Mesh has  74107  elements,  37164  points,  and  111270  edges.


In [109]:
Draw(A.V, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [39]:
 A.mesh.GetMaterials()

('core',
 'fill_air',
 'inner_air',
 'glass',
 'soft_polymer',
 'hard_polymer',
 'buffer',
 'Outer')

In [41]:
center = center
radius = .1        
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=10,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


Set NGSolve automatic PML with p= 2  alpha= 5 and thickness=3.333
Using FEAST to search for vector leaky modes in
circle of radius 0.1 centered at  3933.841831856479
assuming not more than 4 modes in this interval
System size: 43674  x  43674   Inverse type: umfpack

SpectralProj: Setting shifted trapezoidal rule on elliptical contour
SpectralProjNGR:   Making resolvent at z = +3933.913 +0.004j
SpectralProjNGR:   Making resolvent at z = +3933.771 +0.004j
SpectralProjNGR:   Making resolvent at z = +3933.771 -0.004j
SpectralProjNGR:   Making resolvent at z = +3933.913 -0.004j

=========== Starting FEAST iterations ===========
Trying with 4 vectors:

 ITERATION 1 with 4 vectors and 4 left vectors
   During kernel cleaning abs(g):
   [6.0e-03 1.6e-03 2.3e-07 1.8e-04]
   Real part of computed eigenvalues:
   [3933.69868668 3933.84168014 3933.84197205 3934.03246831]
   Imag part of computed eigenvalues:
   [ 5.19856041e-07  8.72092021e-09 -1.38914451e-08  2.07945486e-05]
   Relative Hausdorf

In [42]:
betas

array([3488400.20827471+7.57974637e-11j, 3488400.39429874+4.13989461e-10j])

In [43]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

# Both polymer layers, then air for outer region:

In [145]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 30 / scaling
T_hard_polymer = 30 / scaling

T_outer = 30 / scaling
T_buffer = 10 / scaling

n0 = n_air

outer_materials = [

    {'material': 'soft_polymer',
     'n': n_soft_polymer,
     'T': T_soft_polymer,
     'maxh': .3},

    {'material': 'hard_polymer',
     'n': n_hard_polymer,
     'T': T_hard_polymer,
     'maxh': .4},

    {'material': 'buffer',
     'n': n0,
     'T': T_buffer,
     'maxh': 1},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': 2}
]

In [146]:
A = ARF2( poly_core=True, refine=0, outer_materials=outer_materials)


ModeSolver: Checking if mesh has required regions
Mesh has  6306  elements,  3178  points,  and  9483  edges.


In [147]:
Draw(A.index, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [116]:
 A.mesh.GetMaterials()

('core',
 'fill_air',
 'inner_air',
 'glass',
 'soft_polymer',
 'hard_polymer',
 'buffer',
 'Outer')

In [148]:
center = 5.066
radius = .1        
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=1, niterations=10,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


Set NGSolve automatic PML with p= 1  alpha= 5 and thickness=2.000
Using FEAST to search for vector leaky modes in
circle of radius 0.1 centered at  5.066
assuming not more than 4 modes in this interval
System size: 31578  x  31578   Inverse type: umfpack

SpectralProj: Setting shifted trapezoidal rule on elliptical contour
SpectralProjNGR:   Making resolvent at z =  +5.137 +0.004j
SpectralProjNGR:   Making resolvent at z =  +4.995 +0.004j
SpectralProjNGR:   Making resolvent at z =  +4.995 -0.004j
SpectralProjNGR:   Making resolvent at z =  +5.137 -0.004j

=========== Starting FEAST iterations ===========
Trying with 4 vectors:

 ITERATION 1 with 4 vectors and 4 left vectors
   During kernel cleaning abs(g):
   [1.2e-02 4.9e-03 2.8e-13 5.9e-15]
   Real part of computed eigenvalues:
   [5.06553166 5.06577348 6.01863061 6.17882183]
   Imag part of computed eigenvalues:
   [-7.29015199e-05  1.05050168e-05  9.95358344e-01 -6.61156119e-02]
   Relative Hausdorff distance from prior iterate: 1

In [152]:
Draw(A.V, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [150]:
betas

array([3488400.88212002+1.00743750e-14j, 3488400.21268484-2.18622077e-13j])

In [153]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…